# Final Project: Texture and Fine-grained Classification

### Objective

For my final project, I'm implementing several `tf.keras` layers, model classes, and other utilities that are useful in constructing models for texture recognition and fine-grained classification problems. My work is in a private github repo, which Pooran has access to, and that I plan to make public shortly.

Most of the components in `keras-texture` are usually attached to the final `conv` output of topless ImageNet-pretrained CNNs, but I'm also experimenting with relatively shallow+wide CNN architectures and multi-level pooling. If I come up with something that works well before the end of the quarter, I may include that in the presentation/report, but for now I'm anticipating just using some subset of the current package components with pretrained models.


### Datasets

So far, I have been testing models with the [Describable Textures Dataset](https://www.robots.ox.ac.uk/~vgg/data/dtd/), and to a lesser extent the [Flickr Materials Database](https://people.csail.mit.edu/celiu/CVPR2010/FMD/).


### Code

The README from the repository is included below. Everything in the **Contents** section has been implemented (although I plan to do a bit of API reorganization -- and more thorough testing, since a few of the layers I have yet to achieve anything near published performance).  



# `keras-texture`

### Requirements

- `numpy`
- `scikit-image`
- `keras`>=2.0
- `tensorflow`
- `scikit-learn` (for logistic regression init and SVC)

The TensorFlow requirement is not enforced in `setup.py`, due to ambiguity between `tensorflow` and `tensorflow-gpu`. This package allows CPU or GPU versions, since some functionality (*e.g.*, Fisher vector encoding with pretrained models) shouldn't necessarily require a GPU.

#### Additional requirements: FV-CNN

Use of the Fisher vector CNN class (`texture.fisher.FVCNN`) requires the [cyvlfeat](https://github.com/menpo/cyvlfeat) wrappers for VLFeat, which should be installed using conda: `conda install -c menpo cyvlfeat`, if at all possible. This packages is not required in other `texture` modules, so it is also not explicitly enforced in `setup.py`.

## Contents

### `Encoding` Layer

The residual encoding layer proposed in [Deep TEN: Texture Encoding Network](https://arxiv.org/pdf/1612.02844.pdf) [*CVPR*, 2017]. This `keras` implementation is largely based on the [PyTorch-Encoding](https://github.com/zhanghang1989/PyTorch-Encoding) release by the paper authors.

<p align="center">
  <img src="./docs/images/Encoding-Layer_diagram.png?raw=true" alt="Encoding-Layer diagram"/>
</p>

The layer learns a `KxD` dictionary of codewords (a "codebook"), and codeword assignment `scale` weights. These are used to encode the residuals of an input of shape `NxD` or `HxWxD` with respect to the codewords. Includes optional L2 normalization of output vectors (`True` by default) and dropout (`None` by default). Unlike the `PyTorch-Encoding` version, only the number of codewords `K` needs to be specified at construction time -- the feature size `D` is inferred from the `input_shape`.

### `BilinearModel` Layer

`BilinearModel` is a trainable `keras` layer implementing the weighted outer product of inputs with shape `[(batches,N),(batches,M)]`. The original idea of bilinear modeling for computer vision problems was proposed in [Learning Bilinear Models for Two-Factor Problems in Vision](http://www.merl.com/publications/docs/TR96-37.pdf) [*CVPR*, 1997].

It is used in the `Deep Encoding Pooling Network (DEP)` proposed in [Deep Texture Manifold for Ground Terrain Recognition](https://arxiv.org/abs/1803.10896) [*CVPR*, 2018] to merge the output of an `Encoding` layer with the output of a standard global average pooling, where both features are extracted from `conv` output of the same `ResNet` base. The intuition is that the former represents textures (orderless encoding) and the latter represents spatially structured observations, so that "[the] outer product representation captures a pairwise correlation between the material texture encodings and spatial observation structures."

![DEP-Architecture](./docs/images/DEP_diagram.png)

### `KernelPooling` Layer

Implementation of [Kernel Pooling for Convolutional Neural Networks](https://vision.cornell.edu/se3/wp-content/uploads/2017/04/cui2017cvpr.pdf) [*CVPR*, 2017]. The layer uses the Count Sketch projection to compute a *p*-order Taylor series kernel with learnable composition. As in the paper, composition weights *alpha* are initialized to approximate a Gaussian RBF kernel. The kernel is computed over all local feature vectors `(h_i, w_j)` in the input volume and then average pooled.

<p align="center">
  <img src="./docs/images/kernel_pooling_diagram.png?raw=true" alt="Kernel Pooling"/>
</p>

Construction paramters include `p` (order of the kernel approximation), `d_i` (dimensionality for each order `i>=2`). Output has shape `(batches, 1+C+(p-1)*d_i)`, where `C` is the number of input channels. 

The *gamma* parameter, which determines *alpha* values in the approximation under the assumption of L2-normalized input vectors, can optionally be estimated using a set of training feature vectors.

### Bilinear `pooling`

`bilinearpooling.py` provides a few convenience functions for creating symmetric or asymmetric B-CNN models in Keras with bilinear pooling, as proposed in [Bilinear CNNs for Fine-grained Visual Recognition](http://vis-www.cs.umass.edu/bcnn/docs/bcnn_iccv15.pdf) (*ICCV*, 2015).

`bilinearpooling.pooling`:

- Average pooling of local feature vector outer products in `tensorflow`
- Includes element-wise signed square root and L2 normalization
- If using `combine`, you won't need to reference this explicitly

`bilinearpooling.combine`: 

- Takes two `keras` models `fA` and `fB` with output shapes `(N, H, W, cA)`, `(N, H, W, cB)`
- Maps `[fA.output, fB.output]` to shape `(N, cA, cB)` with `bilinear.pooling`
- Flattens, connects to `softmax` output using a specifiable number of `Dense` layers.
- Returns the resulting `keras.models.Model` instance

##### Usage Notes

If the dimensionality of local feature vectors is 512, and there are `N` classes, the size of a fully-connected classification layer will be very large (`512*512*N=262,144*N`). Using a ResNet-like model may be unfeasible, since this is number is `4x` larger for models with 2018 output channels. 

## `LogReg` initializer

A lot of models based on 2+ order pooling have large output dimensionality. The `Kernel Pooling` and `B-CNN` papers both initialize fully-connected classifier weights using logistic regression, since with random weight initialization, it's pretty difficult to train a layer of this size (especially for moderate to large `N`). I've written the `LogReg` initializer that uses logistic regression to solve for initial weights.
### FV-CNN

The `texture.fisher` module provides the `FVCNN` class for generating Fisher vector encodings from pretrained CNNs using the `cyvlfeat` wrappers for the `VLFeat` C library. A `FVCNN` instance can be constructed with an arbitrary CNN, or with a string specifying one of the supported ImageNet-pretrained models from `keras.applications`. A training set of images is required to generate the Gaussian Mixture Model of local feature vector distribution and train a support vector classifier. The training set can be a batch-style 4D numpy array, or a list of variable-size 3D image arrays.